Import dependencies

In [1]:
import keras as ks
import numpy as np
from keras import Sequential, layers
from random import randrange as rr

Question-Answer Data

In [2]:
Q1 = "What color is a sky?"
Q2 = "Which color sky has?"
A1 = "Sky has a blue and white gradient colors"
A2 = "Sky have a gradient color of blue and white"
A3 = "Blue and white are mostly presented as sky colors"

In [3]:
DATA = [
    Q1, Q2,
    A1, A2, A3
]

Tokenizing function

In [4]:
def tokenizer(data):
    tokens, vocab = {}, set()
    tokenized_data = []
    for sent in data:
        prepare = [i.lower().removesuffix("?") for i in sent.split()]
        tokenized_data.append(prepare)
        for word in prepare: vocab.add(word)
    vocab, vocab_length = [i for i in vocab], len(vocab)
    for word in range(vocab_length):
        tokens[word+1] = vocab[word]
    for sent in tokenized_data:
        for word in range(len(sent)):
            sent[word] = list(tokens.keys())[list(tokens.values()).index(sent[word])]
    return tokens, tokenized_data, vocab_length

In [5]:
TOKENS, TOKENIZED_DATA, VOCAB_LENGTH = tokenizer(DATA)
TOKENS_QUE, TOKENS_ANS = TOKENIZED_DATA[:2], TOKENIZED_DATA[-3:]

Functions to make train samples out of tokens

In [6]:
def conditional_samples(answer_tokens, target_token):
    samples = []
    for token in answer_tokens:
        try: index = token.index(target_token)
        except: KeyError
        finally: pass
        try: samples.append(token[index+1])
        except: KeyError
        finally: pass
    return samples

In [7]:
def sample_generator(question_tokens, answer_tokens):
    max_ans_len = max([len(i) for i in answer_tokens])
    for que in question_tokens:
        for ans in answer_tokens:
            while len(que) < max_ans_len:
                que.append(0)
            for word in range(len(ans)):
                x_samples = []
                x_samples.append(que)
                sent = ans[:word+1]
                while len(sent) < max_ans_len:
                    sent.append(0)
                x_samples.append(sent)
                y_samples = conditional_samples(answer_tokens, ans[word])
                while len(y_samples) < len(answer_tokens):
                    y_samples.append(0)
                yield x_samples, y_samples

In [8]:
x_samples, y_samples = [], []
for x, y in sample_generator(TOKENS_QUE, TOKENS_ANS):
    x_samples.append(x), y_samples.append(y)
x_samples = np.array(x_samples)
y_samples = np.array(y_samples)

IMPORTANT: Shape of models input and output should match shapes of train data

In [9]:
print(x_samples.shape)
print(y_samples.shape)

(52, 2, 9)
(52, 3)


Constructing a model

In [17]:
mdl = Sequential([
    layers.Input(shape=(2, 9,)),
    layers.Conv1D(6, 2, activation="gelu", padding="same"),
    layers.MaxPooling1D(pool_size=1, padding="same"),
    layers.Conv1D(9, 2, activation="gelu", padding="same"),
    layers.MaxPooling1D(pool_size=1, padding="same"),
    layers.Flatten(),
    layers.Dense(9, activation="leaky_relu"),
    layers.Dense(3)
])
mdl.compile(
    optimizer="adam",
    loss="mae",
    metrics=["accuracy"]
)

Training model

In [18]:
mdl.fit(x_samples, y_samples, epochs=1000)

Epoch 1/1000
2/2 [==============================] - 1s 12ms/step - loss: 9.9647 - accuracy: 0.1538
Epoch 2/1000
2/2 [==============================] - 0s 8ms/step - loss: 9.6759 - accuracy: 0.1538
Epoch 3/1000
2/2 [==============================] - 0s 11ms/step - loss: 9.3961 - accuracy: 0.1538
Epoch 4/1000
2/2 [==============================] - 0s 10ms/step - loss: 9.1328 - accuracy: 0.1538
Epoch 5/1000
2/2 [==============================] - 0s 9ms/step - loss: 8.9002 - accuracy: 0.1538
Epoch 6/1000
2/2 [==============================] - 0s 9ms/step - loss: 8.6578 - accuracy: 0.1538
Epoch 7/1000
2/2 [==============================] - 0s 9ms/step - loss: 8.4602 - accuracy: 0.1538
Epoch 8/1000
2/2 [==============================] - 0s 10ms/step - loss: 8.2755 - accuracy: 0.1538
Epoch 9/1000
2/2 [==============================] - 0s 9ms/step - loss: 8.0988 - accuracy: 0.1538
Epoch 10/1000
2/2 [==============================] - 0s 9ms/step - loss: 7.9443 - accuracy: 0.1538
Epoch 11/1000
2

Pulling a random as start of answer

In [19]:
random_start = []
for word in TOKENS_ANS:
    random_start.append(word[0])

De-tokenizing function

In [24]:
def de_tokenizer(query, tokens):
    ans = []
    for token in query[0][1]:
        try: ans.append(tokens[abs(token)])
        except: KeyError
        finally: pass
    return ans

Making a prompt for model

In [106]:
prompt = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0]) #Empty blank
tokenized_question = np.array(TOKENS_QUE[:1][0]) #Take first question
prompt[0] = random_start[rr(len(TOKENS_ANS))] #Take random starting word

Generating answer to question

In [107]:
query = np.array([[tokenized_question, prompt]])
for count in range(8):
    predicted_tokens = mdl.predict(query)[0]
    j = predicted_tokens[rr(3)]
    query[0][1][count+1] = j

1/1 [==============================] - 0s 39ms/step


De-tokenizing and showing answer

In [108]:
answer = de_tokenizer(query, TOKENS)
print(answer)

['sky', 'have', 'white', 'has', 'blue', 'mostly', 'a']
